In [1]:
import os, sys
path='../'
print(path in sys.path)
if not path in sys.path:
    sys.path.append(path)

False


In [1]:
import os
import pandas as pd
import pyutils.gdrive as gdr
import shutil

ModuleNotFoundError: No module named 'pyutils'

## Input/Output Variables

In [3]:
# Constants

week = 9
cohort='b'
prefix = f"cohort-{cohort}"
gpu_instance = False

if gpu_instance:
    instance_type = "g4dn.2xlarge"
    amiarc="amd64"
else:
    instance_type = "c7g.4xlarge"
    amiarc="arm64"

if gpu_instance:
    udcfilename = f"c{cohort}w{week}_gpu.txt"  
    paramsfilename = f"c{cohort}w{week}_gpu.sh"
else:
    udcfilename = f"c{cohort}w{week}.txt"  
    paramsfilename = f"c{cohort}w{week}.sh"    

gsid = '1CutPYEeZ3O5b7MqUIS6XDD6ddmOexKt9pkUI2u3XFNE'
sheetname = 'Form responses 1'

download_path = '/Users/yabebal/Downloads'
aws_bash_path = '/Users/yabebal/My-Works/10academy/Codes/aws_bash'

## Create bucket in aws S3

In [ ]:
!aws s3 mb s3://10ac-{prefix} --profile ustenac

## Load the google sheet 
Google form is shared with trainees to collect public ssh keys

In [9]:
# header = ['time_stamp', 'first_name', 'last_name', 'email', 'ssh_key']


def get_gsheet_df(gid, sheetname='Sheet0'):
    gst = gdr.gsheet(sheetid=gid,fauth='admin-10ac-service.json') 
    return gst.get_sheet_df(sheetname)
    

try:
    dfb6 = get_gsheet_df(gsid, sheetname=sheetname).T
except:
    raise
    #dfb6 = pd.read_csv('b6_trainees_public_ssh_key.csv')

****** using service account: /Users/yabebal/.credentials/admin-10ac-service.json
service account is being used without delegation..


In [5]:
print(dfb6.shape)
dfb6.head()

(33, 7)


,Timestamp,Email address,First Name,Last Name,Group Number Simple,Group Number,"Copy the content of your PUBLIC SSH KEY (open the public key, copy the content, and paste it here)"
1,21/05/2024 10:40:46,derejehinsermu2@gmail.com,Dereje Hinsermu,Senbatu,G6,G2,ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQDH/SOvlB...
2,21/05/2024 10:50:09,hillary6k@gmail.com,Hillary,Kipkemoi,G4,G4,ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIO9fRAg33B...
3,21/05/2024 11:06:45,mikimare21@gmail.com,Michael,George,G3,G3,ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABgQDANJsrx8...
4,21/05/2024 11:37:44,eyerusad12@gmail.com,Eyerusalem,Admassu,G3,G1,ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQC1NQZJjr...
5,21/05/2024 11:55:15,johnteshe13@gmail.com,Yohanes Teshome,Kebede,G1,G1,ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABgQDR4lR5aS...


In [10]:
dfb6['Group Number'].unique()

array(['G2', 'G4', 'G3', 'G1', 'G6', 'G5', 'G7', 'Tutor'], dtype=object)

## Generate user and ssh files
To automate the creation of linux users, the script requires a file of the following format 
```csv
username1, permission(root)  # this user has root permission
username,                    # this user does not have root permission
```

To give SSH access to each users in the machine, we need to add their public ssh keys in s3 folder 

In [11]:
# iterate through each record of dataframe and create a directory inside ./credentials using df.first_name and 
#df.last_name then create a file inside the directory with the name of df.first_name and df.last_name and 
#write the content of df.ssh_key inside the file

colmap = {'ssh_key':'Copy the content of your PUBLIC SSH KEY (open the public key, copy the content,  and paste it here)'}

def create_user_file(df, gname=''):
    nokrow = 0
    dfg = df.groupby('Group Number')
    
    def func2(row):
        return f"{len(row), type(row)}"
    
    def func(row):
        #x, y = row['First Name'], row['Last Name']
        x, y = row[0], row[1]
        
        try:
            x = x.lower().strip().split()[0]
        except:
            x = ""
        try:
            y = y.lower().strip().split()[0]
        except:
            y = ""
            
        u = ""
        if x:
            u += x
        if y:
            u += "_" + y
            
        if x or y:
            return u
        else:
            return pd.nan
                
    def get_tutors():
        group = 'Tutor'
        gdf = df[df['Group Number'] == 'Tutor']
        df_user_name = pd.DataFrame()        
        df_user_name['user_name'] = gdf[['First Name','Last Name']].apply(lambda row:  func(row), axis=1).dropna().drop_duplicates()     
        df_user_name['access'] = 'root'
        
        #create folder and file
        file_name = f'{aws_bash_path}/compute/user/{prefix}-w{week}-{group.lower()}.txt'
        print('writing to file', file_name)
        df_user_name.to_csv(file_name, header=False, index=False, sep=',')
                
        return df_user_name                
    #
    dftutors = get_tutors()
    if gname == 'Tutor':
        return dftutors
    
    for group, gdf in dfg:  
        print('group', group)
        gdf = gdf.fillna('')
        
        if gname and group != gname:
            print('Skipping group = {group} as gname={gname} ...')
            continue
        

                        
        df_user_name = pd.DataFrame()        
        df_user_name['user_name'] = gdf[['First Name','Last Name']].apply(lambda row:  func(row), axis=1).dropna().drop_duplicates()     
        df_user_name['access'] = 'root'
        df_user_name = pd.concat([df_user_name, dftutors])
        print(df_user_name)
        
        #create folder and file
        file_name = f'{aws_bash_path}/compute/user/{prefix}-w{week}-{group.lower()}.txt'
        print('writing to file', file_name)
        df_user_name.to_csv(file_name, header=False, index=False, sep=',')
        
def create_ssh_akeys_files(df):
        
    # Try to remove the tree; 
    try:
        shutil.rmtree(f'{download_path}/{prefix}-credentials')
    except OSError as e:
        print("Error: %s - %s." % (e.filename, e.strerror))
        
    nokrow = 0
    for index, row in df.iterrows():
        
        #read cols
        user_name = ""
        try:
            first_name = row['First Name'].lower().strip().split()[0]
            user_name += first_name
        except:
            first_name = ""
        try:            
            last_name = row['Last Name'].lower().strip().split()[0]
            user_name += '_' + last_name
        except:
            last_name = ""
        
        if len(user_name)==0:
            continue
                    
        ssh_key = row[colmap.get('ssh_key')].strip()
        
        #create folder and file
        folder_name = f'{download_path}/{prefix}-credentials/{user_name}'
        file_name = folder_name + '/authorized_keys'
           
        #write 
        try:
            os.makedirs(folder_name,exist_ok=True)
            
            if os.path.exists(file_name):
                print(f'File {file_name} already exists. Appending a second ssh key to the file')
                with open(file_name, 'a') as f:
                    f.write(ssh_key)
            else:
                print(f'Adding ssh key to file {file_name} ..')
                with open(file_name, 'w') as f:
                    f.write(ssh_key)
                    
            nokrow += 1
        except: # FileExistsError:
            print(f'Error creating directory {folder_name} or writing to file {file_name}')
            
    print(f'Successfully completed {nokrow} outof {df.shape[0]} rows')

In [12]:
create_user_file(dfb6)
#!rm -rf /Users/yabebal/My-Works/10academy/Codes/aws_bash/compute/user/cohort-a*.txt

writing to file /Users/yabebal/My-Works/10academy/Codes/aws_bash/compute/user/cohort-b-w9-tutor.txt
group G1
             user_name access
4   eyerusalem_admassu   root
5       yohanes_kebede   root
8       mistir_nigusse   root
10      getachew_abebe   root
13   abdelrahman_ahmed   root
24       emtinan_osman   root
29      natnael_melese   root
writing to file /Users/yabebal/My-Works/10academy/Codes/aws_bash/compute/user/cohort-b-w9-g1.txt
group G2
              user_name access
1        dereje_senbatu   root
9          abraham_teka   root
28  temesgen_gebreabzgi   root
24        emtinan_osman   root
29       natnael_melese   root
writing to file /Users/yabebal/My-Works/10academy/Codes/aws_bash/compute/user/cohort-b-w9-g2.txt
group G3
          user_name access
3    michael_george   root
12     addisu_alemu   root
19  abubeker_shamil   root
23    sheila_murugi   root
24    emtinan_osman   root
29   natnael_melese   root
writing to file /Users/yabebal/My-Works/10academy/Codes/aws_bash

In [8]:
create_ssh_akeys_files(dfb6)

Adding ssh key to file /Users/yabebal/Downloads/cohort-b-credentials/dereje_senbatu/authorized_keys ..
Adding ssh key to file /Users/yabebal/Downloads/cohort-b-credentials/hillary_kipkemoi/authorized_keys ..
Adding ssh key to file /Users/yabebal/Downloads/cohort-b-credentials/michael_george/authorized_keys ..
Adding ssh key to file /Users/yabebal/Downloads/cohort-b-credentials/eyerusalem_admassu/authorized_keys ..
Adding ssh key to file /Users/yabebal/Downloads/cohort-b-credentials/yohanes_kebede/authorized_keys ..
Adding ssh key to file /Users/yabebal/Downloads/cohort-b-credentials/daisy_cherono/authorized_keys ..
Adding ssh key to file /Users/yabebal/Downloads/cohort-b-credentials/bethelhem_mebratu/authorized_keys ..
Adding ssh key to file /Users/yabebal/Downloads/cohort-b-credentials/mistir_nigusse/authorized_keys ..
Adding ssh key to file /Users/yabebal/Downloads/cohort-b-credentials/abraham_teka/authorized_keys ..
Adding ssh key to file /Users/yabebal/Downloads/cohort-b-credential

In [9]:
!aws s3 cp ~/Downloads/{prefix}-credentials/ s3://10ac-{prefix}/credentials/ --recursive --profile ustenac
#print(f"~/Downloads/{prefix}-credentials/ s3://10ac-{prefix}/credentials/ --recursive --profile ustenac")

upload: ../../../../../Downloads/cohort-b-credentials/bethelhem_mebratu/authorized_keys to s3://10ac-cohort-b/credentials/bethelhem_mebratu/authorized_keys
upload: ../../../../../Downloads/cohort-b-credentials/daisy_cherono/authorized_keys to s3://10ac-cohort-b/credentials/daisy_cherono/authorized_keys
upload: ../../../../../Downloads/cohort-b-credentials/addisu_alemu/authorized_keys to s3://10ac-cohort-b/credentials/addisu_alemu/authorized_keys
upload: ../../../../../Downloads/cohort-b-credentials/getachew_abebe/authorized_keys to s3://10ac-cohort-b/credentials/getachew_abebe/authorized_keys
upload: ../../../../../Downloads/cohort-b-credentials/gilbert_tcheugoue/authorized_keys to s3://10ac-cohort-b/credentials/gilbert_tcheugoue/authorized_keys
upload: ../../../../../Downloads/cohort-b-credentials/abraham_teka/authorized_keys to s3://10ac-cohort-b/credentials/abraham_teka/authorized_keys
upload: ../../../../../Downloads/cohort-b-credentials/fatai_abdusalam/authorized_keys to s3://10ac

## ec2-compute parameters 

In [13]:
setupnvidea = 'true' if gpu_instance else 'false'

config_file = '''
####where conda is installed
PYTHON_DIR=/opt/miniconda

####root bucket
BUCKET="10ac-cohort-%s"

####credential root folder
CRED_ROOT_FOLDER="$BUCKET/credentials"

####credential folders to copy
CRED_FOLDERS=""


###jupyter notebook folder
NOTEBOOK_FOLDER="/home"

###features to install or setup
mounts3=true
addusers=true
setupconda=true
setupjhub=true
setupjnb=true
setupdocker=true
setupnvidea=%s

'''%(cohort, setupnvidea)

with open(f'../compute/configs/{udcfilename}', 'w') as f:
    f.write(config_file)

In [15]:
param_file = '''
# == key variable ==
cohort=%s
week=%s

if [ -z $group ]; then
   echo "please pass group number as environment variable e.g. export group=1"
   exit 0
fi   
group=${group}

name="group${group}"  #it is also a tag
dns_namespace="g${group}.10academy.org"
service="ec2"

# == AWS CLI profile ==
profile="ustenac"
ssmgittoken="git_token_tenx"
gitaccountname="10xac"

# == control variables 
team="C%straining"
iam_users="fiker natnael" #provide the iam ds user as array here

echo "name=$name, profile=$profile"

# == define s3 path ==
s3bucket="10ac-cohort-${cohort}"
s3root="s3://${s3bucket}"

# == defines what to install ==
udcfile=%s # comment out if you don't want to install compute packages
USERS_FILE="cohort-${cohort}-w${week}-g${group}.txt"
echo "Using: group=${group}, udcfile=${udcfile}, userfile=${USERS_FILE}"

# == often change ==
amiarc=%s #"arm64"    #nvidea
amios="ubuntu"
amifordocker=false

TYPE="%s"   # instance type


# == security
IAM="B4EC2Role" 


# ==  set it once and seldom change ==
KEY="itrain-team-useast1"     # EC2 key pair name
COUNT=1         # how many instances to launch
EBS_SIZE=200    # root EBS volume size (GB)

#=== networking ===
#SG="sg-0606253fdd87db25e"  # (trainees_cluster)
#vpc="vpc-06cf87345b7d5fa44" #  (10xtraining) 
#subnetId="subnet-02990182ba1ce1a9f" # (training-subnet-1) 
#region="eu-west-1"

SG="sg-055b3aff51724bb3e"  # (ssh-only)
vpc="vpc-e7d8659d"  #(us-east1-training-vpc)
subnetId="subnet-ae247fa1"
region="us-east-1"
'''%(cohort, week, cohort.upper(), udcfilename, amiarc, instance_type)

with open(f'../params/{paramsfilename}', 'w') as f:
    f.write(param_file)

## Git commit and push commands

In [16]:
commit = f'''cd  {aws_bash_path}  
git add -u * 
git add compute/user/{prefix}-w{week}-g?.txt 
git add compute/configs/{udcfilename}
git add params/{paramsfilename}
git commit -m "add {prefix} week {week} credentials" 
git push
'''
print(commit)

cd  /Users/yabebal/My-Works/10academy/Codes/aws_bash  
git add -u * 
git add compute/user/cohort-b-w9-g?.txt 
git add compute/configs/cbw9.txt
git add params/cbw9.sh
git commit -m "add cohort-b week 9 credentials" 
git push

